In [3]:
import sys
# Tested module
sys.path.append("/home/simon/Documents/REPO/Exact_laws")


In [20]:
from exact_laws.el_calc_mod.grids import div_on_incgrid, incgrid, grid
from exact_laws.el_calc_mod.datasets import load
import numpy as np

In [21]:
list_prim = [(2,3,4)]
list_sec = [(1,3,4),(3,3,4),(2,2,4),(2,4,4),(2,3,3),(2,3,5)]
nb_sec_by_dirr = 1
N = [256,256,256]
c = [1,2,3]
incgrid = incgrid.IncGrid(grid.Grid(N,N,c), [1,1,1], ['lz','lperp','listperp'],{},'cls')
flux = [np.zeros((1,3)),np.zeros((6,3))]
flux[0][0] = [10,11,12]
flux[1][0] = [1,2,3]
flux[1][1] = [1,4,5]
flux[1][2] = [2,2,3]
flux[1][3] = [4,2,5]
flux[1][4] = [2,3,5]
flux[1][5] = [4,2,5]
quantities = {'flux':flux}
dataset = load(grid = grid.Grid(N=[1,6],coords={'listprim':list_prim,'listsec':list_sec,'nb_sec_by_dirr':1}), params={},quantities=quantities)

In [13]:
output = div_on_incgrid(incgrid,dataset)
output

{'div_flux': array([0.])}

In [24]:
flux = [np.zeros((1,3)),np.zeros((6,3))]
flux[0][0] = [10,11,12]
flux[1][0] = [10,200,3000]
flux[1][1] = [20,300,4000]
flux[1][2] = [30,400,5000]
flux[1][3] = [40,500,6000]
flux[1][4] = [50,600,7000]
flux[1][5] = [60,700,8000]
quantities = {'flux':flux}
dataset = load(grid = grid.Grid(N=[1,6],coords={'listprim':list_prim,'listsec':list_sec,'nb_sec_by_dirr':1}), params={},quantities=quantities)

In [25]:
output = div_on_incgrid(incgrid,dataset)
output

{'div_flux': array([555.])}

In [ ]:
output = {}
list_flux = []
for k in dataset_terms.quantities:
    if k.startswith("flux"):
        output["div_" + k] = np.zeros((len(list_prim)))
        term = dataset_terms.quantities[k]
        list_flux.append(k)

for ind, vect_prim in enumerate(list_prim):
    point_sec = [[], [], []]
    for dirr in range(3):
        for i in range(-nb_sec_by_dirr, nb_sec_by_dirr + 1):
            if not i == 0:
                vect = list(vect_prim)
                vect[dirr] = (
                    (vect[dirr] + i)
                    - (N[dirr] * ((vect[dirr] + i) >= (N[dirr] / 2)))
                    + (N[dirr] * ((vect[dirr] + i) <= (-N[dirr] / 2)))
                )
                vect = tuple(vect)
                loc = -1
                try:
                    index = list_prim.index(vect)
                    loc = 0
                except:
                    index = list_sec.index(vect)
                    loc = 1
                point_sec[dirr].append((loc, index))
    for t in list_flux:
        div_point = 0
        if len(point_sec[0]) == 0:
            div_point += np.nan
        else:
            for dirr in range(3):
                values = []
                for i in len(point_sec[dirr]):
                    values.append(dataset_terms.quantities[t][point_sec[dirr][i][0]][point_sec[dirr][i][1]])
                div_point += cdiff(
                    tab = values,
                    length_case=c[0],
                    precision=nb_sec_by_dirr * 2,
                    period=False,
                    point=True,
                )
        output["div_" + t][ind] = div_point
